In [1]:
import sys
import os
import json

In [2]:
parameters = dict(
    
    #### Shall existing plots be skipped?
    skip_existing = True,
    
    #### Paths
    # Paths for evalaution statistics
    data_path_processed_ref = '/g/data/er4/exv563/hydro_projections/data/evaluation/AWAP', # path of evaluation statistics for historical reference
    data_path_processed_sim = '/g/data/er4/exv563/hydro_projections/data/evaluation/ISIMIP_AWAP/climate_inputs', # path of evaluation statistics for bias corrected climate data
    
    # Paths to raw daily data (not used at the moment)
    # path_daily_ref = '/g/data/er4/data/CLIMATE/#VAR#', # path of original daily data, #VAR# will be replaced by the variable name (for historical reference)
    # path_daily_sim = '/g/data/wj02/COMPLIANT/HMINPUT/output/AUS-5/BoM/BoM-#GCM#/historical/r1i1p1/r240x120-ISIMIP2b-AWAP/v1/day/#VAR#', # path of original daily data, #VAR# will be replaced by the variable name
    
    # Output path
    plot_path = '/g/data/er4/exv563/hydro_projections/plots/data_evaluation/ISIMIP_AWAP/climate_inputs', # where shall plots be saved?
    
    #### General settings
    evaluation_year_start = 1976,
    evaluation_year_end = 2005,
    name_ref = 'awap', # name of reference dataset, used for finding files
    name_sim_prefix = 'isimip', # name of dataset to evaluate, used for finding files
    
    #### GCMs - which GCMs should be plotted
    gcms = ['ACCESS1-0','CNRM-CM5','GFDL-ESM2M','MIROC5'],
    # for those plot types  where applicable (climatology, PDF, CDF), create an additional plot with ALL GCMS in one? Select False, if only creating plots for one GCM.
    include_all_gcms_plot = False,
    
    #### Seasons - for ALL-SEASONS plots, which seasons should be included? (bias maps, PDFs, CDFs, spatial correlation)
    seasons = ['annual', 'DJF', 'MAM', 'JJA', 'SON'], # 
    
    #### Variables - which variables should be included
    vars = ['rain_day','temp_min_day','temp_max_day','wind','solar_exposure_day'],
    units = {'rain_day':'mm','temp_min_day':'°C','temp_max_day':'°C', 'wind':'m/s','solar_exposure_day':'MJ/m2'}, # used for adjusting legends in plots
    # what are the variable names in file names in the reference data set?
    ref_vars = {'rain_day':'rain_day','temp_min_day':'temp_min_day','temp_max_day':'temp_max_day',
                   'wind':'wind','solar_exposure_day':'solar_exposure_day'},
    # what are the variable names inside the netcdf in the reference data set?
    ref_vars_in_nc = {'rain_day':'rain_day','temp_min_day':'temp_min_day','temp_max_day':'temp_max_day',
                   'wind':'wind','solar_exposure_day':'solar_exposure_day'},
    # what are the variable names in file names in the bias corrected climate dataset?
    sim_vars = {'rain_day':'pr','temp_min_day':'tasmin','temp_max_day':'tasmax',
                   'wind':'sfcWind','solar_exposure_day':'rsds'},
    # what are the variable names inside the netcdf in the bias corrected climate dataset?
    sim_vars_in_nc = {'rain_day':'pr','temp_min_day':'tasmin','temp_max_day':'tasmax',
                         'wind':'sfcWind','solar_exposure_day':'rsds'},
    
    #### Masks
    # mask to focus only on AWRA land areas and remove AWAP areas of low confidence
    mask_file = '/g/data/er4/exv563/hydro_projections/data/spatial/awra_awap_mask/awra_awap_mask.nc',
    mask_var_name_in_nc = 'mask',
    
    # NRM super cluster mask
    region_file = '/g/data/er4/exv563/hydro_projections/data/spatial/NRM_regions/NRM_super_clusters_nc/NRM_super_clusters.nc',
    region_var = 'NRM_super_cluster',
    region_meta_data = '/g/data/er4/exv563/hydro_projections/data/spatial/NRM_regions/NRM_super_clusters_nc/NRM_super_clusters_meta.csv',
    region_meta_data_id_column = 'FID',
    region_meta_data_label_column = 'label',
    
    # Which NRM super clusters should be plotted? default: AU, region code 0, 1, 2, 3
    region_codes_to_use = ['AU', 0, 1, 2, 3],
    # region_codes_to_use_spatio_temporal = [0, 1, 2, 3], # use less memory, therefore only NRM regions - not used at the moment

    #### Point locations - coordinates
    point_locations = {'Melbourne': {'lat': -37.769525, 'lon': 145.008552},
               'Sydney': {'lat': -33.782566, 'lon': 150.942748},
               'Darwin': {'lat': -12.381293, 'lon': 130.915446},
               'Perth': {'lat': -31.793321, 'lon': 116.108832}},
    
    #### Plot specific settings
    
    # Bias maps - which statistics to plot?
    bias_maps_statistics = {'rain_day':['sum','pctl05', 'pctl10', 'pctl95', 'pctl90'],
                            'temp_min_day':['mean','pctl05', 'pctl10', 'pctl95', 'pctl90'],
                            'temp_max_day':['mean','pctl05', 'pctl10', 'pctl95', 'pctl90'],
                            'wind':['mean','pctl05', 'pctl10', 'pctl95', 'pctl90'],
                            'solar_exposure_day':['mean','pctl05', 'pctl10', 'pctl95', 'pctl90']},
    
    # Climatologies - which statistics to plot?
    climatologies_statistics = {'rain_day':['sum','pctl05','pctl95'],
                             'temp_min_day':['mean','pctl05','pctl95'],
                             'temp_max_day':['mean','pctl05','pctl95'],
                             'wind':['mean','pctl05','pctl95'],
                             'solar_exposure_day':['mean','pctl05','pctl95']},
    
    # Spatial PDFs - which statistics to plot?
    pdf_spatial_statistics = {'rain_day':['sum'],
                             'temp_min_day':['mean','min'],
                             'temp_max_day':['mean','max'],
                             'wind':['mean'],
                             'solar_exposure_day':['mean']},
    
    # Temporal PDFs - which statistics to plot?
    pdf_temporal_statistics = {'rain_day':['sum'],
                             'temp_min_day':['mean','min'],
                             'temp_max_day':['mean','max'],
                             'wind':['mean'],
                             'solar_exposure_day':['mean']},
    
    # Point PDFs - which statistics to plot?
    point_pdf_statistics = {'rain_day':['sum'],
                             'temp_min_day':['mean','min'],
                             'temp_max_day':['mean','max'],
                             'wind':['mean'],
                             'solar_exposure_day':['mean']},
    
    # Fourier diagram - which statistics to plot?
    # fourier_time_aggregations = ['daily', 'monthly']
    fourier_time_aggregations = ['monthly']
)
    

In [3]:
with open('config.json', 'w') as fp:
    json.dump(parameters, fp, indent=True)

In [5]:
# open the config file with:
# with open('config.json', 'r') as fp:
#     parameters = json.load(fp)